In [1]:
import pandas as pd
import numpy as np
import pulp
from itertools import chain, combinations
import statistics as stats
import matplotlib.pyplot as plt
import seaborn as sns


counties_df = pd.read_csv('data/uscounties_geo.csv', header=0)
counties_df = counties_df[counties_df['state_id'] == 'WA'].reset_index(drop=True)
counties_df

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population
0,King,King,King County,53033,WA,Washington,47.4902,-121.8052,2240876
1,Pierce,Pierce,Pierce County,53053,WA,Washington,47.0241,-122.1046,910225
2,Snohomish,Snohomish,Snohomish County,53061,WA,Washington,48.0475,-121.6975,820024
3,Spokane,Spokane,Spokane County,53063,WA,Washington,47.6207,-117.4040,531477
4,Clark,Clark,Clark County,53011,WA,Washington,45.7792,-122.4825,496494
5,Thurston,Thurston,Thurston County,53067,WA,Washington,46.9258,-122.8332,290642
6,Kitsap,Kitsap,Kitsap County,53035,WA,Washington,47.6130,-122.6716,273072
7,Yakima,Yakima,Yakima County,53077,WA,Washington,46.4571,-120.7385,255151
8,Whatcom,Whatcom,Whatcom County,53073,WA,Washington,48.8259,-121.7199,224533
9,Benton,Benton,Benton County,53005,WA,Washington,46.2398,-119.5112,204551


In [2]:
districts_df= pd.read_csv('data/county_districts.csv', header=0)
districts_df['county']=districts_df['county'].str.lstrip(' ').str.rstrip(' ')
counties_df['county']=counties_df['county'].str.lstrip(' ').str.rstrip(' ')
counties_df=counties_df.merge(districts_df, on='county', how='left')
counties_df

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,District
0,King,King,King County,53033,WA,Washington,47.4902,-121.8052,2240876,7
1,Pierce,Pierce,Pierce County,53053,WA,Washington,47.0241,-122.1046,910225,10
2,Snohomish,Snohomish,Snohomish County,53061,WA,Washington,48.0475,-121.6975,820024,1
3,Spokane,Spokane,Spokane County,53063,WA,Washington,47.6207,-117.4040,531477,5
4,Clark,Clark,Clark County,53011,WA,Washington,45.7792,-122.4825,496494,3
5,Thurston,Thurston,Thurston County,53067,WA,Washington,46.9258,-122.8332,290642,10
6,Kitsap,Kitsap,Kitsap County,53035,WA,Washington,47.6130,-122.6716,273072,6
7,Yakima,Yakima,Yakima County,53077,WA,Washington,46.4571,-120.7385,255151,4
8,Whatcom,Whatcom,Whatcom County,53073,WA,Washington,48.8259,-121.7199,224533,1
9,Benton,Benton,Benton County,53005,WA,Washington,46.2398,-119.5112,204551,4


In [3]:
## white only census
census_df = pd.read_csv('data/census_demographics.csv', header=0)
census_df['WA_MALE']=pd.to_numeric(census_df['WA_MALE'])
census_df['WA_FEMALE']=pd.to_numeric(census_df['WA_FEMALE'])
census_df['WHITE_POP']=census_df['WA_MALE']+census_df['WA_FEMALE']
census_df = census_df.groupby(['COUNTY', 'CTYNAME'])[['TOT_POP', 'WHITE_POP']].sum().reset_index()
census_df['CTYNAME']=census_df['CTYNAME'].str.lstrip(' ').str.rstrip(' ')
census_df

,COUNTY,CTYNAME,TOT_POP,WHITE_POP
0,1,Adams County,165642,145704
1,3,Asotin County,179178,165888
2,5,Benton County,1675328,1501572
3,7,Chelan County,636014,591962
4,9,Clallam County,621528,537626
5,11,Clark County,4075954,3474526
6,13,Columbia County,31884,28776
7,15,Cowlitz County,890726,806168
8,17,Douglas County,347996,320036
9,19,Ferry County,58178,44344


In [4]:
# the strings arent joining but I can sort and join on the index
counties_df=counties_df.sort_values(by=['county']).reset_index(drop=True)
census_df=census_df.sort_values(by=['CTYNAME']).reset_index(drop=True)
counties_df=counties_df.merge(census_df, left_index=True, right_index=True, how='left')
for i,j in zip( census_df['CTYNAME'].to_list(), counties_df['county'].to_list()):
    print(i, '  ', j)

counties_df

Adams County    Adams
Asotin County    Asotin
Benton County    Benton
Chelan County    Chelan
Clallam County    Clallam
Clark County    Clark
Columbia County    Columbia
Cowlitz County    Cowlitz
Douglas County    Douglas
Ferry County    Ferry
Franklin County    Franklin
Garfield County    Garfield
Grant County    Grant
Grays Harbor County    Grays Harbor
Island County    Island
Jefferson County    Jefferson
King County    King
Kitsap County    Kitsap
Kittitas County    Kittitas
Klickitat County    Klickitat
Lewis County    Lewis
Lincoln County    Lincoln
Mason County    Mason
Okanogan County    Okanogan
Pacific County    Pacific
Pend Oreille County    Pend Oreille
Pierce County    Pierce
San Juan County    San Juan
Skagit County    Skagit
Skamania County    Skamania
Snohomish County    Snohomish
Spokane County    Spokane
Stevens County    Stevens
Thurston County    Thurston
Wahkiakum County    Wahkiakum
Walla Walla County    Walla Walla
Whatcom County    Whatcom
Whitman County    Whit

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,District,COUNTY,CTYNAME,TOT_POP,WHITE_POP
0,Adams,Adams,Adams County,53001,WA,Washington,46.9834,-118.5606,20353,4,1,Adams County,165642,145704
1,Asotin,Asotin,Asotin County,53003,WA,Washington,46.1918,-117.2030,22285,5,3,Asotin County,179178,165888
2,Benton,Benton,Benton County,53005,WA,Washington,46.2398,-119.5112,204551,4,5,Benton County,1675328,1501572
3,Chelan,Chelan,Chelan County,53007,WA,Washington,47.8692,-120.6190,78508,8,7,Chelan County,636014,591962
4,Clallam,Clallam,Clallam County,53009,WA,Washington,48.0493,-123.9280,76727,6,9,Clallam County,621528,537626
5,Clark,Clark,Clark County,53011,WA,Washington,45.7792,-122.4825,496494,3,11,Clark County,4075954,3474526
6,Columbia,Columbia,Columbia County,53013,WA,Washington,46.2975,-117.9078,3969,5,13,Columbia County,31884,28776
7,Cowlitz,Cowlitz,Cowlitz County,53015,WA,Washington,46.1932,-122.6810,109457,3,15,Cowlitz County,890726,806168
8,Douglas,Douglas,Douglas County,53017,WA,Washington,47.7361,-119.6918,42622,8,17,Douglas County,347996,320036
9,Ferry,Ferry,Ferry County,53019,WA,Washington,48.4703,-118.5166,7198,5,19,Ferry County,58178,44344


In [5]:
del counties_df['county']
counties_df=counties_df.reset_index(names='county')
counties_df

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,District,COUNTY,CTYNAME,TOT_POP,WHITE_POP
0,0,Adams,Adams County,53001,WA,Washington,46.9834,-118.5606,20353,4,1,Adams County,165642,145704
1,1,Asotin,Asotin County,53003,WA,Washington,46.1918,-117.2030,22285,5,3,Asotin County,179178,165888
2,2,Benton,Benton County,53005,WA,Washington,46.2398,-119.5112,204551,4,5,Benton County,1675328,1501572
3,3,Chelan,Chelan County,53007,WA,Washington,47.8692,-120.6190,78508,8,7,Chelan County,636014,591962
4,4,Clallam,Clallam County,53009,WA,Washington,48.0493,-123.9280,76727,6,9,Clallam County,621528,537626
5,5,Clark,Clark County,53011,WA,Washington,45.7792,-122.4825,496494,3,11,Clark County,4075954,3474526
6,6,Columbia,Columbia County,53013,WA,Washington,46.2975,-117.9078,3969,5,13,Columbia County,31884,28776
7,7,Cowlitz,Cowlitz County,53015,WA,Washington,46.1932,-122.6810,109457,3,15,Cowlitz County,890726,806168
8,8,Douglas,Douglas County,53017,WA,Washington,47.7361,-119.6918,42622,8,17,Douglas County,347996,320036
9,9,Ferry,Ferry County,53019,WA,Washington,48.4703,-118.5166,7198,5,19,Ferry County,58178,44344


In [6]:
# Washington has 10 districts
max_district = 10
counties_df['TOT_POP'] = counties_df['TOT_POP'].astype("int")
counties_df['WHITE_POP'] = counties_df['WHITE_POP'].astype("int")
counties_df['lat'] = counties_df['lat'].astype("float")
counties_df['lng'] = counties_df['lng'].astype("float")

max_district = 10
max_population = (counties_df['population'].sum() / max_district) * 1.5
statewide_white_percentage = counties_df['WHITE_POP'].sum() / counties_df['TOT_POP'].sum()

counties_lat_Dic = dict(zip(counties_df.county, counties_df.lat))
counties_Long_Dic = dict(zip(counties_df.county, counties_df.lng))
counties_Pop_Dic = dict(zip(counties_df.county, counties_df.TOT_POP))
counties_WhitePop_Dic = dict(zip(counties_df.county, counties_df.WHITE_POP))



In [7]:
county_adjacency = {}
with open("data/county_adjacency.txt", 'r', encoding='ISO-8859-1') as file:
    lines = file.readlines()

for line in lines:
    parts = line.strip().split('\t')
    if len(parts) < 4:
        continue
    county_fips = parts[1]
    if county_fips.startswith('53'):
        county_name = parts[0].replace('"', '')
        adjacent_county_fips = parts[3]
        adjacent_county_name = parts[2].replace('"', '')
        if county_name not in county_adjacency:
            county_adjacency[county_name] = []
        if adjacent_county_fips.startswith('53'):
            county_adjacency[county_name].append(adjacent_county_name)

# Create the adjacency matrix
sorted_counties = sorted(county_adjacency.keys())
adjacency_matrix = pd.DataFrame(0, index=sorted_counties, columns=sorted_counties, dtype=int)
for county, neighbors in county_adjacency.items():
    for neighbor in neighbors:
        adjacency_matrix.at[county, neighbor] = 1
        adjacency_matrix.at[neighbor, county] = 1

# Adjust the indices to match county names
county_index_to_name_map = counties_df['county'].to_dict()

index_to_county_map = {index: county for index, county in enumerate(counties_df['county'])}


In [8]:
adjacency_matrix=adjacency_matrix.sort_index().reset_index(drop=True)
adjacency_matrix.columns = counties_df['county'].to_list()
adjacency_matrix

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [10]:
from gurobipy import GRB, Model

def compactness(district):
    lat_list = [counties_lat_Dic[county] for county in district]
    long_list = [counties_Long_Dic[county] for county in district]
    
    lat_sd = stats.stdev(lat_list)
    long_sd = stats.stdev(long_list)
    
    return lat_sd + long_sd

def total_pop(district):
    pop_list = [counties_Pop_Dic.get(county, 0) for county in district]
    return sum(pop_list)

def white_pop_percentage(district):
    white_pop_list = [counties_WhitePop_Dic.get(county, 0) for county in district]
    total_pop_list = [counties_Pop_Dic.get(county, 0) for county in district]
    white_pop = sum(white_pop_list)
    total_pop = sum(total_pop_list)
    return (white_pop / total_pop) if total_pop > 0 else 0

min_len = 2
max_len = 5  
counties = counties_df['county'].tolist()
possible_districts = list(chain.from_iterable(combinations(counties, i) for i in range(min_len, max_len + 1)))

redistrict_model = Model("Redistricting Model")

# create the variables as before
x = redistrict_model.addVars(possible_districts, vtype=GRB.BINARY, name="district")

# set the objective as before
redistrict_model.setObjective(
    sum([compactness(district) * x[district] for district in possible_districts]) +
    sum([abs(white_pop_percentage(district) - statewide_white_percentage) * x[district] for district in possible_districts]),
    GRB.MINIMIZE
)

# add the constraints as before
redistrict_model.addConstr(sum([x[district] for district in possible_districts]) == max_district, "Maximum_number_of_districts")

for district in possible_districts:
    for county1 in district:
        for county2 in district:
            if county1 != county2:
                if adjacency_matrix.at[county1, county2] == 0:
                    redistrict_model.addConstr(x[district] == 0)

# optimize the model
redistrict_model.optimize()

# print the status and the solution
if redistrict_model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    for v in redistrict_model.getVars():
        if v.x > 0:
            print(v.varName, v.x)
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7R13 Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 48 physical cores, 96 logical processors, using up to 32 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [14]:
from gurobipy import GRB, Model

def compactness(district):
    lat_list = [counties_lat_Dic[county] for county in district]
    long_list = [counties_Long_Dic[county] for county in district]
    
    lat_sd = stats.stdev(lat_list)
    long_sd = stats.stdev(long_list)
    
    return lat_sd + long_sd

def total_pop(district):
    pop_list = [counties_Pop_Dic.get(county, 0) for county in district]
    return sum(pop_list)

def white_pop_percentage(district):
    white_pop_list = [counties_WhitePop_Dic.get(county, 0) for county in district]
    total_pop_list = [counties_Pop_Dic.get(county, 0) for county in district]
    white_pop = sum(white_pop_list)
    total_pop = sum(total_pop_list)
    return (white_pop / total_pop) if total_pop > 0 else 0

min_len = 2
max_len = 5  
counties = counties_df['county'].tolist()
possible_districts = list(chain.from_iterable(combinations(counties, i) for i in range(min_len, max_len + 1)))

redistrict_model = pulp.LpProblem("Redistricting Model", pulp.LpMinimize)

x = pulp.LpVariable.dicts('district', possible_districts, cat=pulp.LpBinary)

redistrict_model += (
    sum([compactness(district) * x[district] for district in possible_districts]) +
    pulp.lpSum([abs(white_pop_percentage(district) - statewide_white_percentage) * x[district] for district in possible_districts])
), "Objective"


redistrict_model += sum([x[district] for district in possible_districts]) == max_district, "Maximum_number_of_districts"

for district in possible_districts:
    for county1 in district:
        for county2 in district:
            if county1 != county2:
                if adjacency_matrix.at[county1, county2] == 0:
                    redistrict_model += x[district] == 0
                    
solver = pulp.GUROBI_CMD(msg=True)
redistrict_model.solve(solver)
    
if redistrict_model.status == pulp.LpStatusOptimal:
    print("Optimal solution found!")
else:
    print("No optimal solution found. Status:", pulp.LpStatus[redistrict_model.status])

/home/ubuntu/.local/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


KeyboardInterrupt: 

In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Assuming counties_df and other required dataframes are already defined and loaded as per your snippets

# Your existing functions
def compactness(district):
    lat_list = [counties_lat_Dic[county] for county in district]
    long_list = [counties_Long_Dic[county] for county in district]
    lat_sd = stats.stdev(lat_list)
    long_sd = stats.stdev(long_list)
    return lat_sd + long_sd

def total_pop(district):
    pop_list = [counties_Pop_Dic.get(county, 0) for county in district]
    return sum(pop_list)

def white_pop_percentage(district):
    white_pop_list = [counties_WhitePop_Dic.get(county, 0) for county in district]
    total_pop_list = [counties_Pop_Dic.get(county, 0) for county in district]
    white_pop = sum(white_pop_list)
    total_pop = sum(total_pop_list)
    return (white_pop / total_pop) if total_pop > 0 else 0

# Helper function to calculate compactness and white population percentage in parallel
def calculate_objective_for_district(district):
    compactness_value = compactness(district)
    white_pop_percent_value = white_pop_percentage(district)
    return compactness_value, white_pop_percent_value

# Initialize the optimization model
redistrict_model = pulp.LpProblem("Redistricting Model", pulp.LpMinimize)
counties = counties_df['county'].tolist()
min_len = 2
max_len = 5
possible_districts = list(chain.from_iterable(combinations(counties, i) for i in range(min_len, max_len + 1)))
x = pulp.LpVariable.dicts('district', possible_districts, cat=pulp.LpBinary)

# Calculate objective values for each district in parallel
objective_compactness = []
objective_white_pop = []
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust the number of workers as needed
    future_to_district = {executor.submit(calculate_objective_for_district, district): district for district in possible_districts}
    for future in as_completed(future_to_district):
        district = future_to_district[future]
        compactness_value, white_pop_percent_value = future.result()
        objective_compactness.append(compactness_value * x[district])
        objective_white_pop.append(abs(white_pop_percent_value - statewide_white_percentage) * x[district])


# Add the objective to the model
redistrict_model += (
    sum(objective_compactness) +
    pulp.lpSum(objective_white_pop)
), "Objective"

# Constraints
# Maximum number of districts
redistrict_model += sum([x[district] for district in possible_districts]) == max_district, "Maximum_number_of_districts"

# Adjacency constraint for each pair of counties in a district
for district in possible_districts:
    for county1 in district:
        for county2 in district:
            if county1 != county2:
                if adjacency_matrix.at[county1, county2] == 0:
                    redistrict_model += x[district] == 0

# Each county must be in exactly one district
for county in counties:
    redistrict_model += sum([x[district] for district in possible_districts if county in district]) == 1, f"Must_zone_{county}"

# Solve the model
redistrict_model.solve()

# Output the results
if redistrict_model.status == pulp.LpStatusOptimal:
    print("Optimal solution found!")
    final_output="Optimal solution found!"
else:
    print("No optimal solution found. Status:", pulp.LpStatus[redistrict_model.status])
    final_output="No optimal solution found."



/home/ubuntu/.local/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


NameError: name 'concurrent' is not defined

In [ ]:
# Print the Objective Function
print("Objective Function:")
print(str(redistrict_model.objective))

# Print the Constraints
print("\nConstraints:")
for name, constraint in redistrict_model.constraints.items():
    print(f"{name}: {constraint}")


Objective Function:
1.5361444444047387*district_('Adams',_'Asotin') + 1.6187932848659192*district_('Adams',_'Asotin',_'Benton') + 2.580324390594346*district_('Adams',_'Asotin',_'Chelan') + 4.498958746021727*district_('Adams',_'Asotin',_'Clallam') + 3.360095712393774*district_('Adams',_'Asotin',_'Clark') + 1.125151697095383*district_('Adams',_'Asotin',_'Columbia') + 3.3260778433211473*district_('Adams',_'Asotin',_'Cowlitz') + 2.036904575341122*district_('Adams',_'Asotin',_'Douglas') + 1.939832328140642*district_('Adams',_'Asotin',_'Ferry') + 1.3094760866583948*district_('Adams',_'Asotin',_'Franklin') + 1.128595678016454*district_('Adams',_'Asotin',_'Garfield') + 1.6833213940305545*district_('Adams',_'Asotin',_'Grant') + 3.9916372151318167*district_('Adams',_'Asotin',_'Grays_Harbor') + 3.778720381350424*district_('Adams',_'Asotin',_'Island') + 4.163790270299605*district_('Adams',_'Asotin',_'Jefferson') + 3.0345564322387855*district_('Adams',_'Asotin',_'King') + 3.5627702640850742*distric

In [ ]:
import geopandas as gpd

geojson_path = 'data/washington-state-counties_.geojson'
geojson_gdf = gpd.read_file(geojson_path)
final_data_df['county'] = final_data_df['county'].str.title()
merged_gdf = geojson_gdf.merge(final_data_df, left_on='NAME', right_on='county', how='left')
merged_gdf[['NAME', 'District', 'geometry']]

DriverError: /mnt/data/washington-state-counties_.geojson: No such file or directory

In [ ]:
chosen_districts = [district for district in possible_districts if x[district].value() == 1]

# Step 2: Create a mapping of counties to districts
county_to_district_map = {}
district_number = 1
for district in chosen_districts:
    for county in district:
        county_to_district_map[county] = district_number
    district_number += 1

# Step 3: Update the DataFrame
counties_df['Optimized_District'] = counties_df['county'].map(county_to_district_map)


In [ ]:
import matplotlib.pyplot as plt

# Plot the original districts map
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# The original district information isn't available in the provided data
# If we had it, we would plot it here
# For demonstration purposes, we'll assume the 'District' in the CSV represents the original district
geojson_gdf.plot(ax=ax[0], color='white', edgecolor='black')
merged_gdf.plot(ax=ax[0], column='District', cmap='viridis', legend=True, legend_kwds={'shrink': 0.3})
ax[0].set_title('Original Districts')
ax[0].set_axis_off()

# Plot the optimized districts map
merged_gdf.plot(ax=ax[1], column='Optimized_District', cmap='viridis', legend=True, legend_kwds={'shrink': 0.3})
ax[1].set_title('Optimized Districts')
ax[1].set_axis_off()

plt.show()